## Running OWL2Vec*

In [9]:
from rdflib import Graph
g = Graph()
g.parse("modified_original_data/Merged_GeoFault_modified.ttl", format="turtle")
print("graph has %s statements." % len(g))
g.serialize(destination='modified_original_data/Merged_GeoFault_modified.owl', format='xml')

graph has 3009 statements.


<Graph identifier=Nf68758b5254340bc89ea67ceae423976 (<class 'rdflib.graph.Graph'>)>

In [ ]:
from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download
word2vec_model = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/wikipedia2vec_enwiki_20180420_win10_100d", filename="enwiki_20180420_win10_100d.txt"))
word2vec_model.save("word2vec.model")


KeyboardInterrupt: 

In [ ]:
from owl2vec_star import owl2vec_star


#Parameters:
# ontology_file
# config_file
# uri_doc
# lit_doc
# mix_doc
gensim_model = owl2vec_star.extract_owl2vec_model("./modified_original_data/Merged_GeoFault_modified.owl", "./default.cfg", True, True, True)

output_folder="./cache/output/"

#Gensim format
gensim_model.save(output_folder+"ontology.embeddings")
#Txt format
gensim_model.wv.save_word2vec_format(output_folder+"ontology.embeddings.txt", binary=False)


INFO: Access the ontology ...
ERROR: PROBLEM LOADING the ontology with OWLReady. An OWL 2 compliant ontology is expected in RDF/XML, OWL/XML or NTriples format: 
	<class 'FileNotFoundError'>
	[Errno 2] No such file or directory: './Merged_GeoFault_modified.owl'
INFO: Calculate the ontology projection ...
INFO: Creating ontology graph projection...
INFO: 	Extracting subsumption triples


AttributeError: 'OntologyAccess' object has no attribute 'graph'

## Loading embeddings and getting similarities

In [11]:
from gensim.models import KeyedVectors

#Embedding vectors generated above
model = KeyedVectors.load("./cache/output/ontology.embeddings", mmap='r')
wv = model.wv

vector = wv['throw']  # Get numpy vector of a word
print("Vector for 'throw'")
print(vector)

#cosine similarity
similarity = wv.similarity('throw', 'http://www.semanticweb.org/quy/ontologies/2022/8/untitled-ontology-33#GeoFault:throw')
print(similarity)

similarity = wv.similarity('http://www.semanticweb.org/quy/ontologies/2022/8/untitled-ontology-33#GeoFault:relay', 'relay')
print(similarity)


#Most similar cosine similarity
result = wv.most_similar(positive=['relay', 'throw'])
print(result)

#Most similar entities: cosmul
result = wv.most_similar_cosmul(positive=['relay'])
print(result)

#Get list of keys
keys = wv.index_to_key
print("List of keys")
print(keys)

INFO: loading KeyedVectors object from ./cache/output/ontology.embeddings
INFO: loading wv recursively from ./cache/output/ontology.embeddings.wv.* with mmap=r
INFO: setting ignored attribute cum_table to None
INFO: Word2Vec lifecycle event {'fname': './cache/output/ontology.embeddings', 'datetime': '2025-05-07T16:31:12.046011', 'gensim': '4.3.2', 'python': '3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:51:40) \n[GCC 13.3.0]', 'platform': 'Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39', 'event': 'loaded'}


Vector for 'throw'
[-0.39487416 -0.31630176 -0.16663079  0.35813704 -0.01613964  0.3730667
 -0.01280351  0.10225102 -0.59422374 -0.6355193   0.10196559 -0.29167953
  0.02846472 -0.09813152  0.16219564 -0.38081926 -0.0695428  -0.26509574
 -0.8898334  -0.18305022 -0.03434346 -0.3121131   0.55374825  0.4084217
  0.31990397 -0.04748817  0.47238585 -0.276305    0.14863522 -0.18826623
 -0.26857823 -0.22413822  0.26626664 -0.07654157  0.16036557  0.03019057
 -0.15355662 -0.58316356  0.3237537   0.02848702  0.28932783 -0.18798923
  0.23989037  0.05119241  0.55569893 -0.0238639   0.5287298  -0.01937538
  0.1965178  -0.30927342  0.08756916 -0.3064449  -0.5863684   0.18567316
 -0.00398788 -0.02665281  0.04753711  0.5352311   0.32925674  0.569454
  0.03822102 -0.11855158  0.02989179 -0.04234906 -0.16916348 -0.20221272
  0.27891257 -0.17901073  0.01589837  0.07225776 -0.25742528 -0.09639575
 -0.95362943 -0.3356427   0.17686203  0.31911957 -0.05610082 -0.34310627
 -0.24857731  0.09208968 -0.07774588

In [12]:
from flashtext import KeywordProcessor

keyword_processor = KeywordProcessor()

keyword_processor.add_keywords_from_list(keys)

# keyword_processor.add_keyword('The questions')
# keyword_processor.add_keyword('Starting Point')

sentence = "Protolith Role: a role that is role_of  some amount of rock. It indicates the original rock type of the rock before the rock participate_in some Geological Process."


from nltk.tokenize import MWETokenizer
tokenizer = MWETokenizer()

# Remove punctuation and uppercase
sentence = sentence.replace(".", "").replace(",", "").replace(":", "").replace(";", "").lower()
sentence = sentence.replace("(", "").replace(")", "").replace("[", "").replace("]", "")
sentence = sentence.replace("'", "").replace('"', "").replace("`", "")
sentence = sentence.replace("!", "").replace("?", "").replace("&", "").replace("%", "")
sentence = sentence.replace("$", "").replace("#", "").replace("@", "").replace("^", "")
sentence = sentence.replace("*", "").replace("-", "").replace("+", "").replace("=", "")
sentence = sentence.replace("{", "").replace("}", "").replace("<", "").replace(">", "")
sentence = sentence.replace("|", "").replace("\\", "").replace("/", "").replace("~", "")
sentence = sentence.replace("`", "").replace("´", "").replace("`", "").replace("‘", "")
sentence = sentence.replace("“", "").replace("”", "").replace("'", "").replace("’", "")

keywords_found = keyword_processor.extract_keywords(sentence, span_info=True)
print(keywords_found)

for a in keywords_found:
    tokenizer.add_mwe(sentence[a[1]: a[2]].split())
    print(sentence[a[1]: a[2]])

tokenizer.tokenize(sentence.split())

print(wv['protolith role'].shape)

[('protolith role', 0, 14), ('a', 15, 16), ('role', 17, 21), ('that', 22, 26), ('is', 27, 29), ('role_of', 30, 37), ('some', 39, 43), ('amount of rock', 44, 58), ('it', 59, 61), ('the', 72, 75), ('rock', 85, 89), ('Type', 90, 94), ('of', 95, 97), ('the', 98, 101), ('rock', 102, 106), ('the', 114, 117), ('rock', 118, 122), ('some', 138, 142), ('geological process', 143, 161)]
protolith role
a
role
that
is
role_of
some
amount of rock
it
the
rock
type
of
the
rock
the
rock
some
geological process
(100,)
